In [3]:
from ptgnn.runtime_config.run_config import fetch_loaders

In [4]:
data_config = {
    'dataset' : {
        'type' : "rs",
        'mask_chiral_tags': True,
        'transformation_mode': 'chienn_tree_basic',
    },
    'loader': {
        'general': {
            'n_neighbors_in_circle': 3,
            'batch_size': 32,
            'num_workers': 0,
        },
        'train': {},
        'test': {},
        'val': {}
    }
}

In [5]:
train_loader, _, val_loader = fetch_loaders(data_config=data_config)

In [6]:
for batch in val_loader:
    display(batch)
    break

DataBatch(x=[2356, 118], edge_index=[2, 6416], edge_attr=[6416, 80], pos=[2356, 6], y=[32], batch=[2356], ptr=[33], ptree=[2356])

# model for one type (Z)

In [7]:
# put through linear layers
# todo: do so only for a  few and not for all as not all needed for this module

In [8]:
k = 3
hidden_dim = 118

In [9]:
import torch

In [8]:
k_list = torch.nn.ModuleList([
    torch.nn.Linear(118, hidden_dim)
    for _ in range(k)
])

In [9]:
# put all elements through linear layers
# todo add masking to only do to relevant layers
elem_z_put_though = [
    k_elem(batch.x)
    for k_elem in k_list
]

In [10]:
elem_z_put_though

[tensor([[ 0.2861, -0.2076,  0.0117,  ...,  0.2689, -0.0708, -0.0571],
         [ 0.0971, -0.0944, -0.0788,  ...,  0.1648,  0.0102, -0.0895],
         [ 0.1436, -0.1071,  0.1052,  ...,  0.0311, -0.0337, -0.1437],
         ...,
         [ 0.3100, -0.0640, -0.0865,  ...,  0.2082, -0.0444, -0.0096],
         [ 0.0092, -0.3440,  0.0048,  ...,  0.1946,  0.0930, -0.0004],
         [ 0.3100, -0.0640, -0.0865,  ...,  0.2082, -0.0444, -0.0096]],
        grad_fn=<AddmmBackward0>),
 tensor([[-0.0540,  0.1426, -0.0411,  ...,  0.0189, -0.0058,  0.3411],
         [ 0.1692,  0.1748, -0.1855,  ...,  0.2441, -0.0910, -0.2534],
         [ 0.1098,  0.0153, -0.1968,  ...,  0.1879, -0.0011, -0.0757],
         ...,
         [-0.1305,  0.3478,  0.0317,  ..., -0.1131,  0.0361,  0.1787],
         [ 0.3895,  0.1442, -0.3329,  ...,  0.0253, -0.1248, -0.1514],
         [-0.1305,  0.3478,  0.0317,  ..., -0.1131,  0.0361,  0.1787]],
        grad_fn=<AddmmBackward0>),
 tensor([[ 0.0561,  0.2315,  0.1396,  ...,  0.15

In [10]:
import json

In [12]:
# extract permutation trees from string
permutation_trees = [
    json.loads(p_string)
    for p_string in batch.ptree
]

In [13]:
permutation_trees

[{'P': [1]},
 {'P': [0, {'Z': [3, 7, 5]}]},
 {'P': [3, {'Z': [0, 5, 7]}]},
 {'P': [2, {'Z': [9]}]},
 {'P': [5, {'Z': [0, 7, 3]}]},
 {'P': [4, {'Z': [14, 19]}]},
 {'P': [7, {'Z': [0, 3, 5]}]},
 {'P': [6]},
 {'P': [9, {'Z': [2]}]},
 {'P': [8, {'Z': [11, 13]}]},
 {'P': [11, {'Z': [8, 13]}]},
 {'P': [10, {'Z': [15, 17]}]},
 {'P': [13, {'Z': [8, 11]}]},
 {'P': [12]},
 {'P': [15, {'Z': [10, 17]}]},
 {'P': [14, {'Z': [4, 19]}]},
 {'P': [17, {'Z': [10, 15]}]},
 {'P': [16]},
 {'P': [19, {'Z': [4, 14]}]},
 {'P': [18, {'Z': [21, 23, 25]}]},
 {'P': [21, {'Z': [18, 25, 23]}]},
 {'P': [20, {'Z': [27, 31, 29]}]},
 {'P': [23, {'Z': [18, 21, 25]}]},
 {'P': [22, {'Z': [26, 35, 33]}]},
 {'P': [25, {'Z': [18, 23, 21]}]},
 {'P': [24]},
 {'P': [27, {'Z': [20, 29, 31]}]},
 {'P': [26, {'Z': [22, 33, 35]}]},
 {'P': [29, {'Z': [20, 31, 27]}]},
 {'P': [28]},
 {'P': [31, {'Z': [20, 27, 29]}]},
 {'P': [30]},
 {'P': [33, {'Z': [22, 35, 26]}]},
 {'P': [32]},
 {'P': [35, {'Z': [22, 26, 33]}]},
 {'P': [34]},
 {'P': [3

In [103]:
len(permutation_trees)

2356

In [9]:
def depth(d):
    if isinstance(d, list):
        return max(map(depth, d))
    if isinstance(d, dict):
        return 1 + (max(map(depth, d.values())) if d else 0)
    return 0

In [10]:
# get number of layers
num_layers = depth(permutation_trees)
num_layers

NameError: name 'permutation_trees' is not defined

In [ ]:
def get_at_depth(d: object, depth: int = 1) -> list:
    if depth == 1:
        # return the values
        return list(d.values())
    else:
        return [
            *get_at_depth(child, depth-1)
            for child in d.values()
        ]

In [ ]:
# get actions to take at level max
actions = [

    for tree in permutation_trees
]

In [ ]:
def get_matrix_layer_at_depth(tree, depth, k):
    if isinstance(tree, int):
        return tree
    elif isinstance(tree, dict):
        key = next(iter(tree.keys()))
        l = get_matrix_layer_at_depth(tree[key], depth-1, k=k)
        if depth == 1:

    else:
        return [
            get_matrix_layer_at_depth(child, depth, k=k)
            for child in tree
        ]

In [ ]:
def get_matrix(tree, depth, prefix: list = []) -> list:
    if isinstance(tree, list):
        return torch.cat([
            get_matrix(child, depth, prefix + [idx])
            for idx, child in enumerate(tree)
        ], dim=-1)

    elif isinstance(tree, dict):
        key = next(iter(tree.keys()))
        return get_matrix(tree[key], depth-1, prefix)

    elif isinstance(tree, int):
        return torch.tensor(prefix + [tree] + [-1]*depth)

    else:
        return [-1]

temp = get_matrix(permutation_trees, depth=num_layers)
temp

In [11]:
def get_matrix(tree, depth, prefix: list = []) -> list:
    if isinstance(tree, list):
        return torch.cat([
            get_matrix(child, depth, prefix + [idx])
            for idx, child in enumerate(tree)
        ], dim=0)

    elif isinstance(tree, dict):
        key = next(iter(tree.keys()))
        return get_matrix(tree[key], depth-1, prefix)

    elif isinstance(tree, int):
        return torch.tensor(prefix + [tree] + [-1]*depth).reshape(1, -1)

    else:
        return [-1]

temp = get_matrix(permutation_trees, depth=num_layers)
temp[:10, :]

NameError: name 'permutation_trees' is not defined

In [12]:
type_dict = {
    "P": 1,
    "Z": 2
}

In [13]:
def get_matrix(tree, depth, idx_prefix: list = [], type_prefix: list = []) -> list:
    if isinstance(tree, list):
        idx_matrix, type_matrix = zip(*[
            get_matrix(child, depth, idx_prefix + [idx], type_prefix)
            for idx, child in enumerate(tree)
        ])
        return torch.cat(idx_matrix, dim=0), torch.cat(type_matrix, dim=0)

    elif isinstance(tree, dict):
        key = next(iter(tree.keys()))
        return get_matrix(tree[key], depth - 1, idx_prefix, type_prefix=type_prefix + [type_dict[key]])

    elif isinstance(tree, int):

        return torch.tensor(idx_prefix + [tree] + [-1] * depth).reshape(1, -1), torch.tensor(type_prefix + [0]*depth).reshape(1, -1)

    else:
        return [-1]

idx_matrix, type_matrix = get_matrix(permutation_trees, depth=num_layers)
display(idx_matrix[:10])
display(type_matrix[:10])

NameError: name 'permutation_trees' is not defined

In [11]:
import torch_geometric

In [15]:
# figuring out how global sum aggregation works
torch_geometric.nn.pool.global_add_pool(torch.tensor([1,2,3,4,5]), torch.tensor([1,2,3,4,5]))

tensor([0, 1, 2, 3, 4, 5])

alright put each colum of matrix to range 0 to ... to use addpool. for lowest part add more elements to thing like chienn. seperate matrix into vectors and do recursively to add up to top. different types after each other with mask then add up or set elements directly.

the minus ones should be handled by pushing them to the rightmost and making that type defaulttype(=tunnel through)

either apply all of this in batching or add to generation part like the permutation tree and then in batching collate accordingly

In [89]:
idx_structure, second_layer_ = torch.unique(idx_matrix[:, :2], dim=0, return_counts=True)
display(idx_structure[:10])
display(second_layer_)

tensor([[0, 0],
        [1, 0],
        [1, 1],
        [2, 0],
        [2, 1],
        [3, 0],
        [3, 1],
        [4, 0],
        [4, 1],
        [5, 0]])

tensor([1, 1, 3,  ..., 1, 3, 1])

In [90]:
from itertools import chain

second_layer = torch.repeat_interleave(second_layer_)
display(second_layer)

tensor([   0,    1,    2,  ..., 4122, 4122, 4123])

tensor([   0,    1,    2,  ..., 4122, 4122, 4123])

In [101]:
second_roller = [
    torch.roll(torch.arange(elem), shifts=1).repeat(elem, 1)
    for elem in second_layer_
]
display(second_roller)

[tensor([[0]]),
 tensor([[0]]),
 tensor([[2, 0, 1],
         [2, 0, 1],
         [2, 0, 1]]),
 tensor([[0]]),
 tensor([[2, 0, 1],
         [2, 0, 1],
         [2, 0, 1]]),
 tensor([[0]]),
 tensor([[0]]),
 tensor([[0]]),
 tensor([[2, 0, 1],
         [2, 0, 1],
         [2, 0, 1]]),
 tensor([[0]]),
 tensor([[1, 0],
         [1, 0]]),
 tensor([[0]]),
 tensor([[2, 0, 1],
         [2, 0, 1],
         [2, 0, 1]]),
 tensor([[0]]),
 tensor([[0]]),
 tensor([[0]]),
 tensor([[0]]),
 tensor([[1, 0],
         [1, 0]]),
 tensor([[0]]),
 tensor([[1, 0],
         [1, 0]]),
 tensor([[0]]),
 tensor([[1, 0],
         [1, 0]]),
 tensor([[0]]),
 tensor([[1, 0],
         [1, 0]]),
 tensor([[0]]),
 tensor([[0]]),
 tensor([[1, 0],
         [1, 0]]),
 tensor([[0]]),
 tensor([[1, 0],
         [1, 0]]),
 tensor([[0]]),
 tensor([[1, 0],
         [1, 0]]),
 tensor([[0]]),
 tensor([[0]]),
 tensor([[1, 0],
         [1, 0]]),
 tensor([[0]]),
 tensor([[2, 0, 1],
         [2, 0, 1],
         [2, 0, 1]]),
 tensor([[0]])

In [93]:
torch.repeat_interleave(torch.tensor([1,2,3,5]))

tensor([0, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3])

In [58]:
_, first_layer = torch.unique(idx_structure[:, :1], dim=0, return_counts=True)
display(first_layer)

tensor([1, 2, 2,  ..., 1, 2, 1])

In [59]:
first_layer_ = torch.tensor(list(chain(*[
    [idx] * elem
    for idx, elem in enumerate(first_layer)
])))
display(first_layer_)

tensor([   0,    1,    1,  ..., 2354, 2354, 2355])

alright... transformation is clear - each time it gets smaller (the -1 needs to be replaced with the value to the left consistently)

and... the thing with the type tensor... let's do it

In [65]:
counter = 0
types = []
for i in first_layer:
    types.append(type_matrix[counter, 0])
    counter += i
types = torch.tensor(types)
display(types)

tensor([1, 1, 1,  ..., 1, 1, 1])

# get rid of -1

In [70]:
temp = (idx_matrix[:, -1] == -1)
idx_matrix[temp, -1] = idx_matrix[temp, -2]
display(idx_matrix)

tensor([[   0,    0,    1,    1],
        [   1,    0,    0,    0],
        [   1,    1,    0,    3],
        ...,
        [2354,    1,    1, 2351],
        [2354,    1,    2, 2353],
        [2355,    0, 2354, 2354]])

# add shifted column according to k

In [ ]:
last_layer[]

# put through embeddings - evtl nicht modulelist und dann cat sondern linear(hidden, k*hidden) und dann reshape

In [71]:
last_layer = idx_matrix[:, -1]
display(last_layer)

tensor([   1,    0,    3,  ..., 2351, 2353, 2354])

In [72]:
dir(last_layer)

['H',
 'T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed_

In [78]:
last_layer.shape

torch.Size([6416])

In [77]:
batch.x[last_layer].shape

torch.Size([6416, 118])

In [85]:
temp1 = torch.stack([
    emb(batch.x[last_layer])
    for idx, emb in enumerate(k_list)
], dim=1)
display(temp1.shape)

torch.Size([6416, 3, 118])

In [ ]:
# sum over the k layers
# then elu
# then linear layer

In [75]:
torch.nn.Linear(3, (3,2))

TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [77]:
selection = temp[0]
print(selection)
print(type(selection))
print(len(selection))

[1]
<class 'list'>
1


In [65]:
[-1]*0

[]

# simplistic version:

In [8]:
k=3

In [15]:
z_layer = torch.nn.ModuleList([
    torch.nn.Linear(118, 118)
    for _ in range(k)
])
z_final_layer = torch.nn.Linear(118,118)
z_elu = torch.nn.ELU()

In [16]:
p_layer = torch.nn.Linear(118, 118)
p_final_layer = torch.nn.Linear(118, 118)

In [17]:
def fetch_or_produce(subtree):
    if isinstance(subtree, int):
        return batch.x[subtree]
    else:
        return fulfill_module(subtree)

In [18]:
def fulfill_module(tree):
    # get node type
    node_type = next(iter(tree.keys()))

    # iterate over indices/trees and replace by data
    data_list = [
        fetch_or_produce(subtree)
        for subtree in tree[node_type]
    ]

    data_list = torch.stack(data_list)

    # put through layer
    if node_type == "P":
        # run through layer for each input
        after_p = p_layer(data_list)
        # sum up
        after_p = torch.sum(after_p, dim=0)
        # put through final linear layer
        after_p = p_final_layer(after_p)

        return after_p

    elif node_type == "Z":
        # run through embedding layers
        after_z = [
            layer(data_list)
            for layer in z_layer
        ]
        # shift
        _idx = torch.arange(data_list.shape[0])
        after_z = [
            embedding[torch.roll(_idx, shifts=idx), :]
            for idx, embedding in enumerate(after_z)
        ]
        after_z = torch.stack(after_z)
        # sum
        after_z = torch.sum(after_z, dim=0)
        # apply ELU
        after_z = z_elu(after_z)
        # layer
        after_z = z_final_layer(after_z)
        # sum
        after_z = torch.sum(after_z, dim=0)
        return after_z


In [19]:
torch.tensor([1,2,3,4])[torch.roll(torch.arange(4), shifts=1)]

tensor([4, 1, 2, 3])

In [20]:
out = torch.stack([
    fulfill_module(ptree)
    for ptree in permutation_trees
])
display(out)
display(out.shape)

NameError: name 'permutation_trees' is not defined

In [21]:
new_node_values = []
for ptree in permutation_trees:
    new_node_values.append(
        fulfill_module(ptree)
    )

NameError: name 'permutation_trees' is not defined

In [12]:
from tqdm import tqdm
import json

In [23]:
device = 'cuda'
z_elu = z_elu.to(device)
z_layer = z_layer.to(device)
z_final_layer = z_final_layer.to(device)

p_layer = p_layer.to(device)
p_final_layer = p_final_layer.to(device)

for train_batch in tqdm(train_loader):
    # to device
    batch = batch.to(device)

    # extract permutation trees from string
    permutation_trees = [
        json.loads(p_string)
        for p_string in batch.ptree
    ]

    new_node_values = [
        fulfill_module(ptree)
        for ptree in permutation_trees
    ]

  1%|          | 9/1721 [00:08<28:02,  1.02it/s]


KeyboardInterrupt: 

# Complicated approach

In [13]:
from ptgnn.model.modules.ptree.complex_ptree_layer import ComplexPtreeLayer
from ptgnn.transform.ptree_matrix import permutation_tree_to_matrix

In [14]:
model = ComplexPtreeLayer(3, 118)

In [19]:
# device = 'cuda'
device = 'cpu'

In [20]:
model = model.to(device)

In [17]:
temp = [
    permutation_tree_to_matrix(train_batch.ptree, 3)
    for train_batch in tqdm(train_loader)
]

100%|██████████| 1721/1721 [03:26<00:00,  8.34it/s]


In [22]:
for train_batch, t in zip(tqdm(train_loader), temp):
    train_batch.idx_matrix, train_batch.type_matrix = t # permutation_tree_to_matrix(train_batch.ptree, 3)

    # to device
    train_batch = train_batch.to(device)

    model(train_batch)

  7%|▋         | 118/1721 [00:27<06:13,  4.29it/s]


KeyboardInterrupt: 

In [12]:
batch.idx_matrix, batch.type_matrix = permutation_tree_to_matrix(batch.ptree, 3)

In [13]:
model(batch)

In [50]:
batch.x.device

device(type='cuda', index=0)